# Dyslexia Homework Assistant Pipeline

In [2]:
# Base Data Science snippet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm_notebook

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Step 1: Image processing

# Step 2: OCR

# Step 3: Hyphenation

In [4]:
sample_text = """ L'apparition du vif ne fait qu'une avec celle de la vie organisée, à la fois parce que la vie ne peut surgir du chaos qu'en apportant en quelque sorte une plus-value de consistance à un ensemble dilapidé de forces et de matériaux ; et à la fois parce que l'énergie nécessaire à cette consistance, l'énergie qui va opérer les densifications, les articulations et assurer le lien, l'énergie qui va tout aussi bien enfler les vides, des fentes, truffer la matière, intercaler les forces, aménager les intervalles qui aèrent et donc cohèrent le vivant, cette énergie ne peut venir que d'une force terrible, aussi ténue soit-elle, qui est le vif."""
sample_text

" L'apparition du vif ne fait qu'une avec celle de la vie organisée, à la fois parce que la vie ne peut surgir du chaos qu'en apportant en quelque sorte une plus-value de consistance à un ensemble dilapidé de forces et de matériaux ; et à la fois parce que l'énergie nécessaire à cette consistance, l'énergie qui va opérer les densifications, les articulations et assurer le lien, l'énergie qui va tout aussi bien enfler les vides, des fentes, truffer la matière, intercaler les forces, aménager les intervalles qui aèrent et donc cohèrent le vivant, cette énergie ne peut venir que d'une force terrible, aussi ténue soit-elle, qui est le vif."

## Tokenization

In [51]:
from nltk.tokenize import wordpunct_tokenize,word_tokenize

In [9]:
sample_tokens = wordpunct_tokenize(sample_text)
sample_tokens[:10]

['L', "'", 'apparition', 'du', 'vif', 'ne', 'fait', 'qu', "'", 'une']

In [52]:
sample_tokens = word_tokenize(sample_text)
sample_tokens[:10]

["L'apparition",
 'du',
 'vif',
 'ne',
 'fait',
 "qu'une",
 'avec',
 'celle',
 'de',
 'la']

## Hyphenation

In [12]:
import pyphen
dic = pyphen.Pyphen(lang='fr')

In [18]:
sample_text2 = "je m'appelle dyslexia, une IA pour aider les enfants dyslexiques à faire leurs devoirs"
dic.inserted(sample_text2)

"je m'ap-pel-le -dys-lexia, u-ne IA -pour ai-der -les en-fants -dys-lexi-ques à -fai-re -leurs -de-voirs"

In [95]:
dic.inserted("anticonstitutionnellement")

'an-ti-cons-ti-tu-tion-nel-le-ment'

In [56]:
def hyphenize_text(text):
    tokens = word_tokenize(text)
    tokens = [dic.inserted(token,hyphen = '|') for token in tokens]
    return tokens

In [57]:
hyphenize_text(sample_text2)

['je',
 "m'ap|pelle",
 'dys|lexia',
 ',',
 'une',
 'IA',
 'pour',
 'ai|der',
 'les',
 'en|fants',
 'dys|lexiques',
 'à',
 'faire',
 'leurs',
 'de|voirs']

# Step 4: Custom dyslexia rendering

Goal: create an html from the hyphenized text with different colors by syllable and using specific dyslexic font

In [87]:
import seaborn as sns
from IPython.display import HTML,display


def make_palette(n,name = "husl",as_hex = True):
    convert_hex = lambda r,g,b : '#%02x%02x%02x' % tuple(map(lambda x : int(x*255),(r,g,b)))
    colors = sns.color_palette(name,n)
    if as_hex:
        colors = [convert_hex(*color) for color in colors]
    return colors

def make_html(text,font = 'Arial Black',font_size = 18,render = True):
    if " " in font:
        font = f"'{font}'"
    html = f"""<p style="font-family:{font};font-size:{font_size}px">{text}</p>"""
    if render:
        return display(HTML(html))
    else:
        return html

In [108]:
def custom_rendering_text(tokens,palette = "colorblind"):
    html = []
    for token in tokens:
        html_token = ""
        syllables = token.split("|")
        if len(syllables) == 1:
            colors = ["#000000"]
        else:
            colors = make_palette(len(syllables),palette)
        for color,syl in zip(colors,syllables):
            html_syl = f"""<span style="color:{color}">{syl}</span>"""
            html_token += html_syl
            
        html.append(html_token)
        
    html = " ".join(html)
    
    return html

def dyslexia_text_pipeline(text,palette = "colorblind",font = "Trebuchet MS",font_size = 18,render = True):
    tokens = hyphenize_text(text)
    html = custom_rendering_text(tokens,palette)
    html = make_html(html,font,font_size,render)
    return html

In [109]:
dyslexia_text_pipeline(sample_text2)

In [110]:
dyslexia_text_pipeline(sample_text)